Ищем коинтегрированные пары

In [1]:
import pandas as pd
from itertools import combinations
import numpy as np
import statsmodels.api as sm

from utils import do_adf_regression, do_aeg_regression, calculate_sharpe_ratio

# Константы
correlation_threshold = 0.95
probability_threshold = 95

open_threshold = 0.75
close_threshold = 0.25

In [2]:
tickers = pd.read_parquet('../data/sp500_stocks.parquet')
tickers.set_index('ticker', inplace=True)
tickers

history = pd.read_parquet('../data/sp500_2023-01-01_2025-01-01.parquet')['Adj Close']
combs = pd.DataFrame(combinations(history.columns, 2), columns=['ticker1', 'ticker2'])
print(len(combs))

126253


In [3]:
combs['correlation'] = combs.apply(lambda x: np.corrcoef(history[x['ticker1']], history[x['ticker2']])[0, 1], axis=1)

combs = combs[combs['correlation'] > correlation_threshold]
combs[['ticker1_company', 'ticker1_sector']] = combs['ticker1'].apply(lambda x: pd.Series(tickers[tickers.index == x][['company', 'sector']].values[0]))
combs[['ticker2_company', 'ticker2_sector']] = combs['ticker2'].apply(lambda x: pd.Series(tickers[tickers.index == x][['company', 'sector']].values[0]))
combs.sort_values(by='correlation', ascending=False, inplace=True)
combs = combs[['ticker1', 'ticker1_company', 'ticker1_sector', 'ticker2', 'ticker2_company', 'ticker2_sector', 'correlation']]
combs

,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation
82888,GOOG,Alphabet Inc. (Class C),Communication Services,GOOGL,Alphabet Inc. (Class A),Communication Services,0.999940
77425,FOX,Fox Corporation (Class B),Communication Services,FOXA,Fox Corporation (Class A),Communication Services,0.998343
113692,NWS,News Corp (Class B),Communication Services,NWSA,News Corp (Class A),Communication Services,0.995214
124467,TRGP,Targa Resources,Energy,WMB,Williams Companies,Energy,0.989555
105661,MCO,Moody's Corporation,Financials,SPGI,S&P Global,Financials,0.989390
...,...,...,...,...,...,...,...
28754,BKNG,Booking Holdings,Consumer Discretionary,V,Visa Inc.,Financials,0.950058
32554,BX,Blackstone Inc.,Financials,NFLX,Netflix,Communication Services,0.950049
86528,HIG,Hartford (The),Financials,RSG,Republic Services,Industrials,0.950035
94285,IT,Gartner,Information Technology,SPG,Simon Property Group,Real Estate,0.950035


Определяем коинтеграция по Augmented Dickey-Fuller

In [4]:
combs['adf_probability'] = combs.apply(lambda x: do_adf_regression(history, x['ticker1'], x['ticker2']), axis=1)
combs

,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation,adf_probability
82888,GOOG,Alphabet Inc. (Class C),Communication Services,GOOGL,Alphabet Inc. (Class A),Communication Services,0.999940,99.0
77425,FOX,Fox Corporation (Class B),Communication Services,FOXA,Fox Corporation (Class A),Communication Services,0.998343,99.0
113692,NWS,News Corp (Class B),Communication Services,NWSA,News Corp (Class A),Communication Services,0.995214,NaN
124467,TRGP,Targa Resources,Energy,WMB,Williams Companies,Energy,0.989555,90.0
105661,MCO,Moody's Corporation,Financials,SPGI,S&P Global,Financials,0.989390,99.0
...,...,...,...,...,...,...,...,...
28754,BKNG,Booking Holdings,Consumer Discretionary,V,Visa Inc.,Financials,0.950058,99.0
32554,BX,Blackstone Inc.,Financials,NFLX,Netflix,Communication Services,0.950049,95.0
86528,HIG,Hartford (The),Financials,RSG,Republic Services,Industrials,0.950035,95.0
94285,IT,Gartner,Information Technology,SPG,Simon Property Group,Real Estate,0.950035,95.0


Определяем коинтеграция по Augmented Engle-Granger

In [5]:
combs['aeg_probability'] = combs.apply(lambda x: do_aeg_regression(history, x['ticker1'], x['ticker2']), axis=1)
combs

,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation,adf_probability,aeg_probability
82888,GOOG,Alphabet Inc. (Class C),Communication Services,GOOGL,Alphabet Inc. (Class A),Communication Services,0.999940,99.0,95.0
77425,FOX,Fox Corporation (Class B),Communication Services,FOXA,Fox Corporation (Class A),Communication Services,0.998343,99.0,99.0
113692,NWS,News Corp (Class B),Communication Services,NWSA,News Corp (Class A),Communication Services,0.995214,NaN,NaN
124467,TRGP,Targa Resources,Energy,WMB,Williams Companies,Energy,0.989555,90.0,NaN
105661,MCO,Moody's Corporation,Financials,SPGI,S&P Global,Financials,0.989390,99.0,99.0
...,...,...,...,...,...,...,...,...,...
28754,BKNG,Booking Holdings,Consumer Discretionary,V,Visa Inc.,Financials,0.950058,99.0,95.0
32554,BX,Blackstone Inc.,Financials,NFLX,Netflix,Communication Services,0.950049,95.0,90.0
86528,HIG,Hartford (The),Financials,RSG,Republic Services,Industrials,0.950035,95.0,90.0
94285,IT,Gartner,Information Technology,SPG,Simon Property Group,Real Estate,0.950035,95.0,90.0


In [6]:
print('All combinations:', len(combs))
combs = combs[(combs['adf_probability'].notna()) | (combs['aeg_probability'].notna())]
combs = combs[(combs['adf_probability'] >= probability_threshold) & (combs['aeg_probability'] >= probability_threshold)]
combs.sort_values(by=['aeg_probability', 'adf_probability', 'correlation'], ascending=False, inplace=True)
print('All combinations after filtering:', len(combs))
combs

All combinations: 1520
All combinations after filtering: 434


,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation,adf_probability,aeg_probability
77425,FOX,Fox Corporation (Class B),Communication Services,FOXA,Fox Corporation (Class A),Communication Services,0.998343,99.0,99.0
105661,MCO,Moody's Corporation,Financials,SPGI,S&P Global,Financials,0.989390,99.0,99.0
112669,NRG,NRG Energy,Utilities,TT,Trane Technologies,Industrials,0.988238,99.0,99.0
96369,JPM,JPMorgan Chase,Financials,KKR,KKR & Co.,Financials,0.987268,99.0,99.0
86955,HLT,Hilton Worldwide,Consumer Discretionary,KKR,KKR & Co.,Financials,0.985304,99.0,99.0
...,...,...,...,...,...,...,...,...,...
32456,BX,Blackstone Inc.,Financials,IBM,IBM,Information Technology,0.951070,95.0,95.0
63037,DVA,DaVita,Health Care,WAB,Wabtec,Industrials,0.951051,95.0,95.0
62897,DVA,DaVita,Health Care,NRG,NRG Energy,Utilities,0.950741,95.0,95.0
86471,HIG,Hartford (The),Financials,NVDA,Nvidia,Information Technology,0.950712,95.0,95.0


In [7]:
live = pd.read_parquet('../data/sp500_2025-01-01_2025-07-01.parquet')['Adj Close']

# calculate_sharpe_ratio(history, "FOX", "FOXA", open_threshold, close_threshold)

combined_index = history.index.union(live.index)
positions = pd.DataFrame(index=combined_index)

if not isinstance(positions.columns, pd.MultiIndex):
    positions.columns = pd.MultiIndex.from_product([[], positions.columns])

def calculate_sharpe_ratios(row, history, live):
    global positions

    ticker1 = row['ticker1']
    ticker2 = row['ticker2']

    training_set = history[[ticker1, ticker2]].copy()
    live_set = live[[ticker1, ticker2]].copy()

    hedge_ration = sm.OLS(training_set[ticker1], training_set[ticker2]).fit().params

    (training_set_sharpe, _, _, training_positions) = calculate_sharpe_ratio(training_set, ticker1, ticker2, hedge_ration.iloc[0], open_threshold, close_threshold)
    (live_set_sharpe, spread_mean, spread_std, live_positions) = calculate_sharpe_ratio(live_set, ticker1, ticker2, hedge_ration.iloc[0], open_threshold, close_threshold)

    combined = pd.concat([training_positions, live_positions])
    positions[training_positions.columns] = combined
    positions = positions.copy()

    return pd.Series((hedge_ration.iloc[0], training_set_sharpe, live_set_sharpe, spread_mean, spread_std))

def to_positions_column(row, names):
    ticker1 = row['ticker1']
    ticker2 = row['ticker2']

    names.append(f"{ticker1}_{ticker2}")

# calculate_sharpe_ratios(combs.iloc[0], history, live)
combs[['hedge_ratio', 'training_set_sharpe', 'live_set_sharpe', 'spread_mean', 'spread_std']] = combs.apply(lambda x: calculate_sharpe_ratios(x, history, live), axis=1)


In [8]:
filtered = combs[(combs['training_set_sharpe'] > 2) & (combs['live_set_sharpe'] > 3.5)]
filtered.sort_values(by=['live_set_sharpe', 'training_set_sharpe'], ascending=False, inplace=True)
print('All combinations after calculating sharpe ratios:', len(filtered))
filtered

All combinations after calculating sharpe ratios: 7


/var/folders/y6/2_hgjshx1x90x58mj93ymhm40000gn/T/ipykernel_6753/1718245924.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered.sort_values(by=['live_set_sharpe', 'training_set_sharpe'], ascending=False, inplace=True)


,ticker1,ticker1_company,ticker1_sector,ticker2,ticker2_company,ticker2_sector,correlation,adf_probability,aeg_probability,hedge_ratio,training_set_sharpe,live_set_sharpe,spread_mean,spread_std
76798,FIS,Fidelity National Information Services,Financials,FITB,Fifth Third Bancorp,Financials,0.959536,95.0,95.0,2.016683,2.104048,4.458189,-3.151110,6.959764
113594,NVR,"NVR, Inc.",Consumer Discretionary,SHW,Sherwin-Williams,Materials,0.970597,99.0,95.0,24.272809,2.001566,4.351045,-1060.447313,411.071682
15800,ANET,Arista Networks,Information Technology,KKR,KKR & Co.,Financials,0.977130,99.0,99.0,0.725637,2.515258,4.071896,1.709350,5.850847
22533,AXP,American Express,Financials,BSX,Boston Scientific,Health Care,0.973786,99.0,95.0,3.143274,2.136788,3.845732,-30.124183,18.848069
59695,DLR,Digital Realty,Real Estate,PNR,Pentair,Industrials,0.952991,99.0,95.0,1.763587,3.051837,3.830390,-4.070781,5.885739
117371,PHM,PulteGroup,Consumer Discretionary,RSG,Republic Services,Industrials,0.965800,95.0,95.0,0.580929,2.116178,3.667558,-32.522092,13.128317
110124,MSI,Motorola Solutions,Information Technology,WELL,Welltower,Real Estate,0.982721,99.0,99.0,3.645609,2.601459,3.638210,-99.401668,47.540381


In [9]:
pairs = np.array(filtered[['ticker1', 'ticker2', 'hedge_ratio', 'spread_mean', 'spread_std']])
pairs

array([['FIS', 'FITB', 2.0166827200214286, -3.151110176118245,
        6.9597644043492535],
       ['NVR', 'SHW', 24.272809211211204, -1060.4473125112788,
        411.0716821441784],
       ['ANET', 'KKR', 0.7256367343071851, 1.7093501932763027,
        5.850847048822151],
       ['AXP', 'BSX', 3.1432737823458745, -30.12418279425897,
        18.848068648721352],
       ['DLR', 'PNR', 1.763587452017863, -4.070780631403686,
        5.885738577298644],
       ['PHM', 'RSG', 0.5809290009621927, -32.52209195692768,
        13.128317290994794],
       ['MSI', 'WELL', 3.6456087944374405, -99.40166817061889,
        47.54038119697217]], dtype=object)

In [10]:
names = []
filtered.apply(lambda x: to_positions_column(x, names), axis=1)

positions = positions[names]
positions

FIS_FITB      NVR_SHW     ANET_KKR     AXP_BSX     DLR_PNR      \
                FIS FITB     NVR SHW     ANET KKR     AXP BSX     DLR PNR   
Date                                                                        
2023-01-03       -1    1       1  -1        0   0       0   0      -1   1   
2023-01-04       -1    1       1  -1        1  -1       0   0      -1   1   
2023-01-05        0    0       1  -1        1  -1       0   0      -1   1   
2023-01-06        0    0       1  -1        1  -1       0   0      -1   1   
2023-01-09        0    0       1  -1        1  -1       0   0      -1   1   
...             ...  ...     ...  ..      ...  ..     ...  ..     ...  ..   
2025-06-24        0    0       0   0        0   0       0   0       0   0   
2025-06-25        0    0       0   0        0   0       0   0       0   0   
2025-06-26        0    0       0   0        0   0       0   0       1  -1   
2025-06-27        0    0       0   0        0   0       0   0       1  -1   
2025-06-30        0    0       0   0        0   0       0   0       0   0   

           PHM_RSG     MSI_WELL       
               PHM RSG      MSI WELL  
Date                                  
2023-01-03       1  -1       -1    1  
2023-01-04       1  -1       -1    1  
2023-01-05       1  -1       -1    1  
2023-01-06       1  -1       -1    1  
2023-01-09       1  -1       -1    1  
...            ...  ..      ...  ...  
2025-06-24       0   0        1   -1  
2025-06-25       0   0        1   -1  
2025-06-26       0   0        1   -1  
2025-06-27       0   0        1   -1  
2025-06-30       0   0        1   -1  

[624 rows x 14 columns]

In [11]:
count_nonzero = (positions != 0).sum(axis=1) / 2
print(f"Mean amount of open positions {count_nonzero.mean()} with std: {count_nonzero.std()}")


Mean amount of open positions 3.233974358974359 with std: 1.5075198044530649


Что нужно сделать дальше:
 - понять как расчитывается PnL чтобы добавить туда транзакционные расходы
 - посчитать все Sharpe ratio с учетом транзакционных расходов
 - отфильтровать какой-то набор пар с которыми буду работать
 - на основе массива positions подготовить сигналы